## Creating galaxy density maps using the method described in Duivenvoorden et al. 2016 (this method is based on Darvish et al. 2015)


![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>)
   

In [2]:
import numpy as np

import os
from astropy.io import fits

from astropy import units as u
from astropy.coordinates import SkyCoord

import pprint
from scipy.special import erf
from scipy.integrate import simps, trapz
from scipy.stats.mstats import gmean

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline  

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

from matplotlib.colors import LinearSegmentedColormap

cdict = {'red': ((0., 0.8, 0.8),
                 (0.08, 0.8, 0.8),
                 (0.3, 0.0, 0.0),
                 (1, 1, 1)),
         'green': ((0., 1.0, 1.0),
                 (0.08, 0.8, 0.8),
                   (0.3, 0.0, 0.0),
                   (0.73, 0.0, 0.0),
                   (1, 0, 0)),
         'blue': ((0., 0.8, 0.8),
                 (0.08, 1, 1),
                  (0.45, 0.5, 0.5),
                  (0.65, 0.0, 0.0),
                  (1, 0, 0))}

my_cmap = LinearSegmentedColormap('my_colormap',cdict,120)
name_field = 'COSMOS'

## The Codes uses the HELP masterlist to select objects with photometric redshifts. 
### The data needs to be homogeneous, to avoid peaks in the density maps caused by deeper data. 
### A mass cut (or IRAC/K-band) can be used to select galaxies with accurate photometric redshifts. 
### In this example we assume a Gaussian shape of the redshift PDF, the full redshift PDFs can be found
### soon at HeDaM to get a more accurate (but slower) result. 


In [3]:
loc = ''
hdulist = fits.open(loc+'cosmos_allz.fits') # test file whihc contains the phot-z in COSMOS
img = hdulist[1].data
cols = hdulist[1].columns
hdulist.close()
z_med = np.array(img['zm_eazy'])
z_low = np.array(img['l68_eazy']) # 1s lower limit
z_high = np.array(img['u68_eazy']) # 1s uper limit
ra = np.array(img['RA_2'])
dec = np.array(img['DEC_2'])
mag1 = np.array(img['m_vista_ks']) # K band data
mag2 = np.array(img['m_wircam_ks'])
mag3 = np.array(img['m_ukidss_k'])

# make the selection, in this user case a simple K band < 24 selection.
use = (z_med >= 0.01) & (z_high <= 3.5) & ( (mag1 < 24.0) | (mag2 < 24) | (mag3 < 24)) 

err = (z_high[use] - z_low[use])/2. # Gaussian assumption for PDF
z_med = z_med[use]
ra = ra[use]
dec = dec[use]
print(np.size(ra))
print(np.mean(err/(1+z_med)))


length = 3900 # create a grid between z = 0.1 and z = 4
rd = 3
z_grid = np.linspace(0.1,4.0, length+1) 
idx = np.arange(length+1)

zmin=0
zmax=1
z_slice_low = []
z_slice_high = []
z_slice_mean= []
# create redshift slics with a width of 2 times the median error
while(z_grid[zmax] < 4): 
    use = (z_med > z_grid[zmin]) & (z_med <= z_grid[zmax])
    if np.size(err[use]) < 200:
        zmax = zmax + 1
    else:            
        med = np.median(err[use])
        if 2*med < (z_grid[zmax]-z_grid[zmin]):
            z_slice_low = np.append(z_slice_low,z_grid[zmin])
            z_slice_high = np.append(z_slice_high,z_grid[zmax])
            mn = z_grid[zmin]+(z_grid[zmax]-z_grid[zmin])/2
            delt =  np.abs(z_grid - round(mn,rd)) == np.min(np.abs(z_grid - round(mn,rd)))
            z_slice_mean = np.append(z_slice_mean,z_grid[delt])
            zmin = idx[delt][0]
        zmax = zmax + 1
        
# saves redshift slices        
DAT = np.array([z_slice_low,z_slice_high]).T
header = 'z_low z_high'
np.savetxt('z_slice_'+name_field+'.cat', DAT, delimiter=" ", fmt='%s',  header= header, newline='\n')        

/Users/Steven/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


310367
0.0632861860603


In [4]:
# Number of mock maps
# 40 were used in the original Duivenvoorden et al. 2016 paper
# this verion uses 5
mock_maps = 5

In [6]:
lengt = 50 #proper kpc, grid for calculating density
for g in range(1,np.size(z_slice_low)):  
  z_l = z_slice_low[g]
  z_h = z_slice_high[g]

  weights = (0.5*(1+erf((z_h-z_med)/(err*np.sqrt(2))))-0.5*(1+erf((z_l-z_med)/(err*np.sqrt(2)))))
  dist = np.array(cosmo.kpc_proper_per_arcmin(z_l/2+z_h/2))
  dist_kpc_dec = (np.max(dec)-np.min(dec))*60*dist # dont use if dec crosses -90,90 
  dist_kpc_ra = (np.max(ra)-np.min(ra))*60*dist # dont use if ra crosses 0, 360
  x_edg = np.linspace(np.min(ra),np.max(ra), ((dist_kpc_ra)/lengt).astype(int)+1) 
  y_edg = np.linspace(np.min(dec),np.max(dec), ((dist_kpc_dec)/lengt).astype(int)+1)

  use_for_speed = (weights > 0.001)

  ra_use_fs = ra[use_for_speed]
  dec_use_fs = dec[use_for_speed]

  x_p = np.zeros(np.size(x_edg) - 1)
  y_p = np.zeros(np.size(y_edg) - 1)
  for i in range(np.size(x_edg) - 1):
    x_p[i] = x_edg[i]/2 + x_edg[i+1]/2
  for i in range(np.size(y_edg) - 1):
    y_p[i] = y_edg[i]/2 + y_edg[i+1]/2

  for n in range(mock_maps+1):#41
    if n > 0:
      random_list = np.int_(np.rint((np.size(ra)-1)*np.random.random([np.size(ra_use_fs)])))
      ra_use_fs = ra[random_list]
      dec_use_fs = dec[random_list]

      
    dens_p_g = np.zeros(np.size(weights[use_for_speed]))
    c1 = SkyCoord(ra_use_fs*u.deg, dec_use_fs*u.deg)

    for i in range(np.size(dens_p_g)):
        weights_use = weights[use_for_speed]
        weights_use[i] = 0
        c2 = SkyCoord(ra_use_fs[i]*u.deg, dec_use_fs[i]*u.deg)
        sep = c1.separation(c2)
        dist_i = sep.arcmin*dist
        K_weigt = np.sum(weights_use/(2*np.pi*500**2)*np.exp(-(dist_i**2)/(2*500**2)))/(np.sum(weights_use))
        dens_p_g[i] = K_weigt
    
    h_use = 500 * np.sqrt(gmean(dens_p_g)/dens_p_g)

    size = ((np.size(x_edg)-1) * (np.size(y_edg)-1))
    val = np.zeros(size)
    dec_use = np.zeros(size)
    ra_use = np.zeros(size)
    count = 0

    sum_w = np.sum(weights[use_for_speed])
    for_f = weights[use_for_speed]/(2*np.pi*h_use**2)
    for i in range(0,np.size(x_p)):
      for j in range(0,np.size(y_p)):
        ra_use[count] = x_p[i]
        dec_use[count] = y_p[j]
        count = count + 1

    h_val = 2*h_use**2

    for i in range(0,size):
        c2 = SkyCoord(ra_use[i]*u.deg, dec_use[i]*u.deg)
        sep = c1.separation(c2)
        dist_i = sep.arcmin*dist

        val[i] = np.sum(for_f*np.exp(-(dist_i**2)/(h_val)))/sum_w
        #print(100*(i+0.0)/size)

      



    S_0, xedges, yedges = np.histogram2d(dec_use,ra_use , bins=(y_edg, x_edg), weights = val)
    DAT = S_0
    header = 'Matix'
    np.savetxt('map%03d'%g+'numb%03d'%n+'_'+name_field+'.cat', DAT, delimiter=" ", fmt='%s',  header= header, newline='\n')

    print(g, 'mean of output  = ', np.mean(S_0), np.sum(S_0))


KeyboardInterrupt: 

### The density map is used in plt_dens.ipynb to assign the desnsity to the galaxies

![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**: Steven Duivenvoorden 

For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).

The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)